In [ ]:
!pip install 'scanpy == 1.9.1'
!pip install 'matplotlib == 3.6'

import scanpy as sc
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import math
from scipy import stats

In [ ]:
def Layer_Cluster_Gene_Exp_Dict_Constructor(all_sup_int_deep, p0_p4_6_p15_annadata, p0_p4_6_p15_names):

  p0_p4_6_p15_layer_exp_mean_dict = {}
  p0_p4_6_p15_layer_exp_standard_deviation_dict = {}

  for anndata, name in zip(p0_p4_6_p15_annadata, p0_p4_6_p15_names):

    curr_timepoint_layer_dict = all_sup_int_deep.get(name)
    all_layer_clust_gene_avg_dict = {}
    all_layer_clust_gene_sd_dict = {}

    for layer in curr_timepoint_layer_dict:

      curr_layer_gene_list = curr_timepoint_layer_dict.get(layer)
      all_clust_gene_avg_dict = {}
      all_clust_gene_sd_dict = {}

      for clust in anndata.obs.E_I_M.cat.categories:

        curr_clust_layer_genes_exp = []

        for gene in curr_layer_gene_list:

          avg_exp = float(anndata.raw[anndata.obs['E_I_M'].isin([clust]),:][:,gene].X.mean(0))
          exp_percentage = float((anndata.raw[anndata.obs['E_I_M'].isin([clust]),:][:,gene].X > 0).mean(0))

          # Percentage * Average Expression
          mean_exp_per_total = avg_exp*exp_percentage

          curr_clust_layer_genes_exp.append(mean_exp_per_total)

        degrees_of_freedom = len(curr_clust_layer_genes_exp) - 1
        curr_clust_layer_genes_standard_deviation = np.std(curr_clust_layer_genes_exp)
        curr_clust_layer_genes_avg = sum(curr_clust_layer_genes_exp)/len(curr_clust_layer_genes_exp)

        all_clust_gene_sd_dict[clust] = (curr_clust_layer_genes_standard_deviation, degrees_of_freedom)
        all_clust_gene_avg_dict[clust] = curr_clust_layer_genes_avg

      all_layer_clust_gene_sd_dict[layer] = all_clust_gene_sd_dict
      all_layer_clust_gene_avg_dict[layer] = all_clust_gene_avg_dict

    p0_p4_6_p15_layer_exp_standard_deviation_dict[name] = all_layer_clust_gene_sd_dict
    p0_p4_6_p15_layer_exp_mean_dict[name] = all_layer_clust_gene_avg_dict

  return p0_p4_6_p15_layer_exp_mean_dict, p0_p4_6_p15_layer_exp_standard_deviation_dict

In [ ]:
def Layer_Specificity_Gene_Expressional_Significance_T_Test(layer_specific_cluster_standard_deviation_dict, layer_specific_cluster_mean_dict, non_layer_specific_cluster_mean_dict):

  timepoints_layer_specific_exp_p_value_dict = {}

  for timepoint in layer_specific_cluster_mean_dict:

    layer_specific_mean_dict = layer_specific_cluster_mean_dict.get(timepoint)
    layer_specific_standard_deviation_dict = layer_specific_cluster_standard_deviation_dict.get(timepoint)

    nls_cluster_gene_mean_dict = non_layer_specific_cluster_mean_dict.get(timepoint).get('All Layer')

    layer_cluster_p_value_dict = {}

    for layer in layer_specific_mean_dict:

      ls_cluster_gene_mean_dict = layer_specific_mean_dict.get(layer)
      ls_cluster_gene_standard_deviation_dict = layer_specific_standard_deviation_dict.get(layer)

      timepoint_cluster_numbers = ls_cluster_gene_mean_dict.keys()

      cluster_p_value_dict = {}

      for cluster in timepoint_cluster_numbers:

        layer_specific_mean = ls_cluster_gene_mean_dict.get(cluster)

        layer_specific_standard_deviation = ls_cluster_gene_standard_deviation_dict.get(cluster)[0]
        layer_specific_degrees_of_freedom = ls_cluster_gene_standard_deviation_dict.get(cluster)[1]

        layer_specific_sample_size = layer_specific_degrees_of_freedom + 1

        non_layer_specific_mean = nls_cluster_gene_mean_dict.get(cluster)

        # T-Test

        net_mean_diff = (layer_specific_mean - non_layer_specific_mean)
        annualized_standard_deviation =  (layer_specific_standard_deviation / math.sqrt(layer_specific_sample_size))

        t_score = (net_mean_diff / annualized_standard_deviation)

        # Left Tailed P-Value
        if t_score < 0:

          p_value = stats.t.sf(abs(t_score), df=layer_specific_degrees_of_freedom)

        # Right Tailed P-Value
        if t_score > 0:

          p_value = stats.t.sf(abs(t_score), df=layer_specific_degrees_of_freedom)

        cluster_p_value_dict[cluster] = p_value

      layer_cluster_p_value_dict[layer] = cluster_p_value_dict

    timepoints_layer_specific_exp_p_value_dict[timepoint] = layer_cluster_p_value_dict

  return timepoints_layer_specific_exp_p_value_dict

In [ ]:
def Plot_Layer_Expression_Distribution(p0_p4_6_p15_layer_exp_dict, p_value_bool, p_val_dict, curr_layer_expression_correlation_dir):

  for timepoint_name in p0_p4_6_p15_layer_exp_dict:

    all_layer_clust_gene_avg_dict = p0_p4_6_p15_layer_exp_dict.get(timepoint_name)

    for layer in all_layer_clust_gene_avg_dict:

      all_clust_gene_avg_dict = all_layer_clust_gene_avg_dict.get(layer)

      cluster_numbers = list(all_clust_gene_avg_dict.keys())
      average_expression_values = list(all_clust_gene_avg_dict.values())

      average_expression_values_dataframe = pd.Series(average_expression_values)

      plt.figure(figsize=(12, 8))
      ax = average_expression_values_dataframe.plot(kind="bar", color = 'maroon')
      if p_value_bool:
        ax.set_title(timepoint_name + '\n'+ layer + ' Layer \n Average Expression vs Clusters')
      else:
        ax.set_title(timepoint_name + '\n'+ layer + ' \n Average Expression vs Clusters')
      ax.set_xlabel("Cluster Numbers")
      ax.set_ylabel("Average expression")
      plt.ylim([0, 1])
      ax.set_xticklabels(cluster_numbers)

      if p_value_bool:

        p_val_values = list(p_val_dict.get(timepoint_name).get(layer).values())

        rects = ax.patches

        for rect, p_val_label in zip(rects, p_val_values):
            height = rect.get_height()
            ax.text(
                rect.get_x() + rect.get_width() / 2, height + .02, "{:.5f}".format(p_val_label), ha="center", va="bottom"
            )
      plt.savefig(curr_layer_expression_correlation_dir+'/'+timepoint_name+'_'+layer+'.png')

      plt.show()